# AMAL TP5
## Réseaux récurrents LSTM, GRU, autres cellules de mémoire


*Comment traiter et générer des séquences de taille variable?*

*Comment prendre en compte des dépendances à plus long terme (Vanishing, exploding, gradients)*

### 1. Générations de séquences de taille variable

Nous considérons que chaque séquence est une phrase (qui peut se terminer par un point, un point d'interrogation ou un point d'exclamation).

Dès lors il faut:
- employer un symbole spécial qui marque la fin d'une séquence EOS. Lors de l'apprentissage, il faut ajouter ce token à chaque séquence pour apprendre à le prédire.
- padder chaque séquence (ajouter un caractère nul (BLANK) autant de fois qu'il est nécessaire afin que les séquences d'un même batch aient toutes la même longueur, ce caractère devra être ignoré lors de l'apprentissage)

##### Q1.
Dans `testloader.py` la classe `TextDataset` est telle que:
- chaque exemple est une phrase
- la taille du dataset est le nbr de phrases ds le corpus
- la phrase renvoyée est sous forme d'une séquence d'entiers, chaque entier codant pr un caractère (application de la fonciton `string2code`)
Cette classe renvoie des exemples de longueur variable. Il faut préciser au DataLoader de quelle manière regrouper les exemples pr construire un batch. C'est le rôle de l'argument `collate_fn` du constructeur d'un DataLoader qui prend une fonction en paramètre.
Définir une fonction `pad_collate_fn` qui prépare un tenseur batch (de taille `longueur x taille du batch` où la longueur = longeur max de la séquence du batch) à partir d'une liste d'exemples du `TextDataset`: elle doit ajouter le code du symbole EOS à la fin de chaque exemple et padder les séquences avec le code du caractère nul. Exécuter `textloader.py`pr vérifier que tout foncitonne bien

De plus, il faut prendre soin de ne pas inclure le padding lorsqu’on calcule le coût ici le maximum de vraisemblance : pour cela, la solution la plus courante est d’utiliser un masque binaire (0 lorsque le caractère est nul, 1 sinon) qui est  multiplié avec les log-probabilités avant
de les additionner (le paramètre `reduce="none"` dans une fonction de coût
(en particulier, pour la cross entropie) permet d’obtenir le coût pour chaque élément plutôt que la moyenne).


In [4]:
%run textloader.py

Chaîne à code :  C'est. Un. Test.
Shape ok
encodage OK
Token EOS ok
Token BLANK ok
Chaîne décodée :  C'est. Un. Test.


/home/nolwenn/AMAL/TME5/tp5/src/textloader.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  samples = [torch.cat((torch.tensor(seq), torch.tensor([EOS_IX]))) for seq in samples]


#### Q2.

Créer votre fonction de coût `maskedCrossEntropy(output, target, padcar) ` dans `tp5.py` qui permet de calculer le coût sans prendre en compte les caracètres nuls en
fonction de la sortie obtenue output, la sortie désirée target et le code de caractère
de padding padcar. Vous ferez attention à n’utiliser aucune boucle pour ce calcul.

In [4]:
import torch

In [3]:
from tp5 import maskedCrossEntropy, RNN


In [3]:
# Dummy data
output_dim = 10
length = 5
batch_size = 3
padcar = 0

# Randomly generated tensors for output and target
output = torch.rand(length, batch_size, output_dim)
target = torch.randint(0, output_dim, (length, batch_size), dtype=torch.long)

# Call maskedCrossEntropy
#lossCecile = maskedCEcile(output, target, padcar)
loss = maskedCrossEntropy(output, target, padcar)

print("Loss:", loss.item())

Loss: 2.1889891624450684


#### Q3. 
Dans `generate.py` implémenter une fonction de génération de façon à générer des séquences jusqu'à rencontrer le caractère EOS (penser à prévoir une taille maximum tout de même). Prévoir de faire de la génération aléatoire dans la distribution obtenue ou déterministe en choisissant le caractère le plus probable à chaque pas de temps.

In [19]:
# Importez toutes les classes nécessaires
from torch import nn
import math
from textloader import string2code, code2string, id2lettre, lettre2id
from tp5 import RNN  # Remplacez par le nom réel de votre module
from generate import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Créez une instance de votre modèle RNN
input_dim = 10  # Remplacez par la dimension appropriée
latent_dim = 64  # Remplacez par la dimension appropriée
output_dim = 10  # Remplacez par la dimension appropriée

rnn_model = RNN(input_dim, latent_dim, output_dim)

# Example: Create dummy embedding and decoder layers
embed_layer = nn.Embedding(num_embeddings = len(lettre2id), embedding_dim = 50, padding_idx = lettre2id['<PAD>']).to(device)
latent_dim = len(lettre2id)
output_dim = len(lettre2id) + 1
decoder_layer = RNN.decode

# Paramètres de la génération
eos_token = 0  # Remplacez par le code de votre caractère EOS

start_sequence = "C'est un test."  # Remplacez par le texte de départ souhaité


In [20]:
# Test generate function

generated_sequence = generate(rnn_model, embed_layer, decoder_layer, eos_token, start_sequence)

print("Generated Beam Sequence:", generated_sequence)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x50 and 10x64)

In [21]:
# Test generate_beam function

beam_k = 3  # Replace with your desired beam size

generated_beam_sequence = generate_beam(rnn_model, embed_layer, decoder_layer, eos_token, beam_k, start=start_sequence)

print("Generated Beam Sequence:", generated_beam_sequence)

ValueError: not enough values to unpack (expected 2, got 1)

In [22]:
alpha_value = 0.7  # Replace with your desired alpha value
nucleus_function = p_nucleus(decoder_layer, alpha_value)

# Example state tensor (replace with your actual state tensor)
example_state = torch.randn(1, latent_dim)

nucleus_probs = nucleus_function(example_state)
print("Nucleus Sampling Probabilities:", nucleus_probs)


TypeError: decode() missing 1 required positional argument: 'h'

Lors de TP4, on a utilisé un encodage onehot pr chaque caractère, suivi d'un module linéaire. Le module `nn.Embedding` de `Torch` permet de combiner ces 2 étapes pr éviter la création (non nécessaire et coûteuse) de vecteurs one hot

### 2. Prise en compte de dépendences lointaines: LSTM et GRU

### 3. Beam Search